In [1]:
import torch
from torch import nn
import torchvision
from torch.utils import data
from torchvision import transforms
# 记录
#from torch.utils.tensorboard import SummaryWriter
import logging
import torch.nn.functional as F


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("////////////////",device)

//////////////// cuda:0


In [3]:
def load_data_mnist(batch_size):  #@save
    trans = [transforms.ToTensor()]
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.MNIST(
        root="./data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.MNIST(
        root="./data", train=False, transform=trans, download=True)
    print('训练集长度：',len(mnist_train))
    print('图片形状：',mnist_train[0][0].shape)
    print('lable：',mnist_train[0][1])
    return (data.DataLoader(mnist_train, batch_size, shuffle=True),
    data.DataLoader(mnist_test, batch_size, shuffle=False))
# 一批图片 256 
batch_size = 256
train_loader,test_loader = load_data_mnist(batch_size=batch_size)

31.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



102.8%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



112.7%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

训练集长度： 60000
图片形状： torch.Size([1, 28, 28])
lable： 5


In [5]:
class letnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.dnn = torch.nn.Sequential(nn.Conv2d(1, 6, kernel_size=5,padding=2),
                          nn.Sigmoid(),
                          nn.AvgPool2d(kernel_size=2, stride=2),
                          nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
                          nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
                          nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
                          nn.Linear(120, 84), nn.Sigmoid(), nn.Linear(84, 10))
    def forward(self, x):
        out = self.dnn(x)
        out= F.softmax(out, dim=1)
        return out

In [6]:
net = letnet().dnn
######## 输入一批256张图 输出256个预测结果  每个预测结果有10个概率
X = torch.rand(size=(256, 1, 28, 28), dtype=torch.float32)

In [7]:
def train(epoch, model):
    train_loss = 0
    train_acc_num = 0
    model.train()
    for feature, lable in train_loader:
        
        feature = feature.to(device)
        lable = lable.to(device)
        optimizer.zero_grad()
        preds = model(feature)
        loss = criterion(preds, lable)
        loss.backward()
        optimizer.step()
        train_acc_num += (preds.argmax(1) == lable).sum()
        train_loss += loss.item() / len(train_loader)
    train_acc = train_acc_num / len(train_loader.dataset)
    print(f'Epoch:{epoch:3} | Train Loss:{train_loss:6.4f} | Train Acc:{train_acc:6.4f}')


In [8]:
def test(model):
    test_acc_num = 0
    model.eval()
    with torch.no_grad():
        for feature, lable in test_loader:
            feature = feature.to(device)
            lable = lable.to(device)
            preds = model(feature)
            test_acc_num += (preds.argmax(1) == lable).sum()
    test_acc = test_acc_num / len(test_loader.dataset)
    print(f'Test Acc:{test_acc:6.4f}')
    return float(test_acc)

In [15]:
def Train(epochs, net, name):
    max = 0
    #writer = SummaryWriter(ScaleName)
    #logging.debug("/////////////////start training name:"+name+"/////////////////")  # 参数msg
    print("////////////////start training name:"+name+"//////////////////")  # 参数msg
    for epoch in range(epochs):
        # 训练
        train(epoch, net)
        # 测试输出
        x = test(net)
        #writer.add_scalar(name,x,epoch)
        # 获取到最大的进行保存和打印
        if max < x:
            max = x
            print("max:", max, "epoch:", epoch)
            torch.save(net.state_dict(), "netmax_{}.pth".format(name))
            #logging.debug("max:"+str(max)+"epoch:"+str(epoch))  # 参数msg
    #writer.close()
    #logging.debug("********************训练完毕****************")  # 参数msg
    print("********************训练完毕****************")


In [18]:
net =letnet()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
Train(25, net,"try5onnx")

////////////////start training name:try5onnx//////////////////
Epoch:  0 | Train Loss:2.2414 | Train Acc:0.1845
Test Acc:0.3776
max: 0.3775999844074249 epoch: 0
Epoch:  1 | Train Loss:1.8963 | Train Acc:0.5938
Test Acc:0.7162
max: 0.7161999940872192 epoch: 1
Epoch:  2 | Train Loss:1.7108 | Train Acc:0.7765
Test Acc:0.8119
max: 0.8118999600410461 epoch: 2
Epoch:  3 | Train Loss:1.6524 | Train Acc:0.8194
Test Acc:0.8337
max: 0.8337000012397766 epoch: 3
Epoch:  4 | Train Loss:1.6319 | Train Acc:0.8357
Test Acc:0.8460
max: 0.8459999561309814 epoch: 4
Epoch:  5 | Train Loss:1.6205 | Train Acc:0.8451
Test Acc:0.8538
max: 0.8537999987602234 epoch: 5
Epoch:  6 | Train Loss:1.6054 | Train Acc:0.8638
Test Acc:0.9075
max: 0.9074999690055847 epoch: 6
Epoch:  7 | Train Loss:1.5537 | Train Acc:0.9214
Test Acc:0.9294
max: 0.9293999671936035 epoch: 7
Epoch:  8 | Train Loss:1.5345 | Train Acc:0.9353
Test Acc:0.9374
max: 0.9373999834060669 epoch: 8
Epoch:  9 | Train Loss:1.5253 | Train Acc:0.9428
Test A